In [ ]:
import pandas as pd
import struct
import datetime
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from dotenv import load_dotenv
import os
from scipy.signal import medfilt
import math

In [ ]:
QUERY = """
select 
    lower(hex(bytes)) as packets_hex,
    bytes as packet
from packets 
where (lower(hex(bytes)) like "aa6400a1%" or lower(hex(bytes)) like "aa5c00f0%") and lower(hex(uuid)) = "610800058d6d82b8614a1c8cb0f8dcc6"
"""

In [ ]:
DATABASE_URL = os.getenv("DATABASE_URL").replace("sqlite://", "sqlite:///../")
df = pd.read_sql(QUERY, DATABASE_URL)

In [ ]:
df["datetime"] = pd.to_datetime(df["packet"].apply(lambda data: struct.unpack('<I', data[11:15])[0]), unit="s")
df = df.sort_values("datetime", ascending=True)

df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time
df["bpm"] = df["packet"].apply(lambda data: data[21])

In [ ]:
def parse_rr(packet: bytes) -> list:
    rr_count = packet[22]
    packet = packet[23:]
    
    rr = []
    for _ in range(4):
        rr_value = struct.unpack('<H', packet[:2])[0]
        packet = packet[2:]
        if rr_value != 0:
            rr.append(rr_value)

    if len(rr) != rr_count:
        raise ValueError("Invalid data")
    return rr

df["rr"] = df["packet"].apply(parse_rr)

In [ ]:
df["stage"] = df["packet"].apply(lambda data: struct.unpack('<I', data[31:35])[0])

In [ ]:
def plot_heart_rate(days, column, plot_count = 2):
    for index, day_data in enumerate(days):
        # Create a figure with secondary y-axes
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Add heart rate trace to primary y-axis
        fig.add_trace(
            go.Scatter(x=day_data['datetime'], y=day_data['bpm'], mode='lines', name='Heart Rate (BPM)', line=dict(color='blue')),
            secondary_y=False,
        )

        # Add x trace to secondary y-axis
        fig.add_trace(
            go.Scatter(x=day_data['datetime'], y=day_data[column], mode='lines', name='X', line=dict(color='orange')),
            secondary_y=True,
        )

        # Update layout for titles and axes
        fig.update_layout(
            title=f"Heart Rate and XYZ from {day_data['datetime'].iloc[0].date()} Noon to Next Day Noon",
            xaxis_title="Time",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )

        # Update x-axis for time formatting
        fig.update_xaxes(tickformat='%H:%M')

        # Update y-axes titles
        fig.update_yaxes(title_text="Heart Rate (BPM)", secondary_y=False)
        fig.update_yaxes(title_text="XYZ Values", secondary_y=True)

        # Show the plot
        fig.show()

        if index == plot_count:
            break

In [ ]:
def filter_noon_to_noon(df):
    # Group the data by date
    days = []
    unique_dates = df['date'].unique()
    
    for date in unique_dates:
        # Define noon of the current day and noon of the next day
        start_noon = pd.Timestamp(datetime.datetime.combine(date, datetime.time(12, 0)))
        end_noon = start_noon + pd.Timedelta(days=1)
        
        # Filter data between the start and end noon
        day_data = df[(df['datetime'] >= start_noon) & (df['datetime'] < end_noon)].copy()
        if not day_data.empty:
            days.append(day_data)

    return days

In [ ]:
def remove_spikes_row(row, window_size=3, threshold=3):
    # Apply median filter
    filtered = medfilt(row, kernel_size=window_size)
    # Identify spikes
    deviation = np.abs(row - filtered)
    is_spike = deviation > threshold * np.std(row)
    # Replace spikes with filtered values
    smoothed = row.copy()
    smoothed[is_spike] = filtered[is_spike]
    return smoothed

In [ ]:
df["stage"] = remove_spikes_row(df["stage"])

In [ ]:
df.loc[df["stage"] < 500_000_000, "stage_category"] = 0
df.loc[(df["stage"] >= 500_000_000) & (df["stage"] < 1000_000_000), "stage_category"] = 1
df.loc[(df["stage"] >= 1000_000_000) & (df["stage"] < 1500_000_000), "stage_category"] = 2
df.loc[df["stage"] > 1500_000_000, "stage_category"] = 3
df["stage_category"] = df["stage_category"].astype("int64")

In [ ]:
days = filter_noon_to_noon(df)

In [ ]:
plot_heart_rate(days, "stage")

In [ ]:
plot_heart_rate(days, "stage_category")

In [ ]:
ACTIVITY_DURATION = 600 * 2

def calculate_heart_rate_stats(row, bpm_df):
    mask = (bpm_df['datetime'] >= row['start']) & (bpm_df['datetime'] <= row['end'])
    relevant_bpm = bpm_df.loc[mask, 'bpm']
    return pd.Series([
        relevant_bpm.mean().round().astype("int64"), 
        relevant_bpm.min().astype("int64"), 
        relevant_bpm.max().astype("int64")
    ])

def calculate_rmssd(window):
    rr_diff = np.diff(window)
    return np.sqrt(np.mean(rr_diff**2)) if len(rr_diff) > 0 else np.nan

def calculate_avg_rmssd(row, df):
    rr_data = df[(df['datetime'] >= row["start"]) & (df['datetime'] <= row["end"])]["rr"].explode().reset_index(drop=True).dropna()
    hrv = rr_data.rolling(300).apply(calculate_rmssd, raw=True).dropna()
    
    return pd.Series([
        hrv.mean().astype("int64"),
        hrv.min().astype("int64"),
        hrv.max().astype("int64")
    ])


def detect_stages(df, stage):
    change = df["stage_category"] == df["stage_category"].shift(1)
    stage_3 = df["stage_category"] == stage
    sleep_starts = df[~change & stage_3]["datetime"].tolist()

    change = df["stage_category"] == df["stage_category"].shift(-1)
    stage_3 = df["stage_category"] == stage
    sleep_ends = df[~change & stage_3]["datetime"].tolist()
    
    stages = pd.DataFrame(list(zip(sleep_starts, sleep_ends)), columns=["start", "end"])

    while True:
        stages["next_start"] = stages["start"].shift(-1)
        stages["duration"] = (stages["end"] - stages["start"]).apply(lambda x: x.total_seconds())
        stages["sleep_diff"] = (stages["next_start"] - stages["end"]).apply(lambda x: x.total_seconds())

        merge_mask = stages["sleep_diff"] < ACTIVITY_DURATION
        
        if not merge_mask.any():
            break
            
        for idx in merge_mask[merge_mask].index:
            if idx + 1 >= len(stages):
                continue

            stages.at[idx, "end"] = stages.at[idx + 1, "end"]
            stages = stages.drop(idx + 1)
        
        stages = stages.reset_index(drop=True)


    stages["duration"] = (stages["end"] - stages["start"]).apply(lambda x: x.total_seconds())
    stages = stages[stages["duration"] >= ACTIVITY_DURATION]

    stages = stages[["start", "end", "duration"]].reset_index(drop=True)
    stages["duration"] = (stages["end"] - stages["start"]).apply(lambda x: round(x.total_seconds() / 3600, 2))
    stages[['avg_bpm', 'min_bpm', 'max_bpm']] = stages.apply(
        calculate_heart_rate_stats, bpm_df=df, axis=1
    )
    stages[["avg_hrv", "min_hrv", "max_hrv"]] = stages.apply(calculate_avg_rmssd, df=df, axis=1)
    return stages

In [ ]:
sleep_df = detect_stages(df, 2)
sleep_df

In [ ]:
exercises = detect_stages(df, 1)
exercises